## Estimating $p_1, p_2$ in queues with parallelizable jobs

###
### Jobs with inherent sizes distributed according to $\text{Exp}(\mu)$ arrive at rate $\lambda$ to a system with $c$ cores.
### Incoming jobs belong to type 1 or 2 (known) with speed-up $s_1(.)$ or $s_2(.)$ parameterized by $p_1, p_2$
### Estimation is based on expressing inherent sizes of jobs as a function of Amdahl parameter
###
### This approach will not work!

In [1]:
import math
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import time

from scipy.optimize import fsolve, minimize
from scipy.integrate import quad
from scipy import linspace, meshgrid, arange, empty, concatenate, newaxis, shape

from collections import deque

In [2]:
def generate_service_times(mu, n):
    service_times = np.random.exponential(1/mu, n)
    return service_times

def generate_arrival_times(lam, n):
    interarrival_times = np.random.exponential(1/lam, n-1)
    arrival_times = np.append([0], np.cumsum(interarrival_times))
    return arrival_times

def generate_job_categories(alpha, n):
    job_categories = np.ones(n) + np.random.binomial(1, alpha, n)
    return job_categories

def allocate_cores(total_cores, n_jobs):
# The following function should come from a learning process
# For now, we assume a simple allocation policy so we can study estimation
    if n_jobs[0] == 0 and n_jobs[1] == 0:
        return np.array([0, 0])
    elif n_jobs[0] == 0:
        return np.array([0, total_cores])
    elif n_jobs[1] == 0:
        return np.array([total_cores, 0])
    else:
        fraction = n_jobs[0]/(n_jobs[0] + n_jobs[1])
        allocation = np.array([fraction*total_cores, (1-fraction)*total_cores])
    return allocation

def speed_up(amdahl_par, n_cores):
    speed_up = 1/(1 - amdahl_par*(1 - 1/max(1, n_cores)))
    return speed_up

In [3]:
class Job:
# Essential characteristics of each job which we would like to track
    def __init__(self):
        self.identity = 0
        self.category = int(0)
        self.arrival_time = 0
        self.departure_time = 0
        self.service_time = 0
        self.residual_service_time = 0
        self.history = [] # List of 2-tuple elements (number of cores allotted, time spent with those cores)

## Simulation

In [4]:
def queue_simulation(amdahl_pars, model_pars, n, arrival_times, service_times, job_categories):
    
    # Unpack model variables
    lam = model_pars[0]
    mu = model_pars[1]
    total_cores = model_pars[2]
    alpha = model_pars[3]
    
    # Preprocessing
    jobs = [Job() for i in range(n)]
    for i in range(n):
        jobs[i].identity = i
        jobs[i].category = job_categories[i]
        jobs[i].arrival_time = arrival_times[i]
        jobs[i].service_time = service_times[i]
        jobs[i].residual_service_time = service_times[i]
    
    ##############################################################################################################################################################
    
    # Data required for future
    events = [] # List of 3-element tuples (Time, Arrival/Departure, Job id)
    n_jobs_in_system_just_after_events = [] # List of 2-element tuples (Number of Type 1 jobs, Number of Type 2 jobs)
    core_allocation_just_after_events = [] # List of 2-element tuples (Number of Type 1 cores, Number of Type 2 cores)
    
    # Simulation variables
    job_ids_in_system = [] # Dynamic list which stores indices of jobs in system at any given time 
    arrived_job_id = -1
    departed_job_id = -1
    event = 'arrival'
    n_jobs_in_system = np.zeros(2)
    current_core_allocation = np.zeros(2)
    current_core_allocation_per_job = np.zeros(2)
    
    current_time = 0
    time_until_next_arrival = 0
    time_until_next_departure = 0
    time_until_next_event = 0
    speed_up_values = np.zeros(2)
    
    ##############################################################################################################################################################
    
    # Queue simulation
    while arrived_job_id < n-1:
        if event == 'arrival':
            arrived_job_id += 1 
            events.append([current_time, 'arrival', arrived_job_id])
            job_ids_in_system.append(arrived_job_id) # Appends arrived job id to the list of jobs in system
            n_jobs_in_system[int(jobs[arrived_job_id].category)-1] += 1
        else:
            events.append([current_time, 'departure', departed_job_id])
            job_ids_in_system.remove(departed_job_id) # Removes departed job id from list of jobs in system
            n_jobs_in_system[int(jobs[departed_job_id].category)-1] -= 1
        
        n_jobs_in_system_just_after_events.append(n_jobs_in_system.copy())
        current_core_allocation = allocate_cores(total_cores, n_jobs_in_system)
        core_allocation_just_after_events.append(current_core_allocation)
        current_core_allocation_per_job = np.array([current_core_allocation[j]/n_jobs_in_system[j] if n_jobs_in_system[j] !=0 else 0 for j in range(2)])
        
        if arrived_job_id != n-1:
            # Time until next arrival
            time_until_next_arrival = arrival_times[arrived_job_id + 1] - current_time
            # Time until next departure
            if job_ids_in_system == []:
                time_until_next_departure = math.inf
            else:
                speed_up_values[0] = speed_up(amdahl_pars[0], current_core_allocation_per_job[0])
                speed_up_values[1] = speed_up(amdahl_pars[1], current_core_allocation_per_job[1])
                expected_times_until_departure = np.array([jobs[i].residual_service_time/speed_up_values[int(jobs[i].category)-1] for i in job_ids_in_system])
                time_until_next_departure = np.min(expected_times_until_departure)
            # Time until next event
            if time_until_next_arrival > time_until_next_departure:
                event = 'departure'
                time_until_next_event = time_until_next_departure
                departed_job_id = job_ids_in_system[np.argmin(expected_times_until_departure)]
                jobs[departed_job_id].departure_time = current_time + time_until_next_event
            else:
                event = 'arrival'
                time_until_next_event = time_until_next_arrival
            
            # Updating system state at time of next event
            for job_id in job_ids_in_system:
                jobs[job_id].residual_service_time -= speed_up_values[int(jobs[job_id].category)-1] * time_until_next_event
                state = [current_core_allocation_per_job[int(jobs[job_id].category)-1], time_until_next_event]
                jobs[job_id].history.append(list(state))
            
            current_time += time_until_next_event
    
    return jobs

## Dataset generation function

In [5]:
def generate_dataset(amdahl_pars, model_pars, n):

    arrival_times = generate_arrival_times(lam, n)
    service_times = generate_service_times(mu, n)
    job_categories = generate_job_categories(alpha, n)
    
    jobs = queue_simulation(amdahl_pars, model_pars, n, arrival_times, service_times, job_categories)
    return jobs

## Estimation

In [6]:
def Log_Likelihood(amdahl_pars, model_pars, jobs):
    print(amdahl_pars)
    
    mu = model_pars[1]
    
    LL = 0
    for job in jobs:
        inherent_size = 0
        for state in job.history:
            inherent_size += speed_up(amdahl_pars[int(job.category)-1], state[0]) * state[1]
        if job.departure_time == 0:
            LL += -mu * inherent_size
        else:
            LL += np.log(mu) -mu * inherent_size
    
    return -LL

In [7]:
def estimation(model_pars, jobs):
    # Initial Estimates
    initial_estimates = np.array([0.5, 0.5])
    # Bounds
    amdahl_par_bounds = [(0.001, 0.999), (0.001, 0.999)]
    # Estimation
    estd_amdahl_pars = minimize(Log_Likelihood, initial_estimates, method = "Nelder-Mead", args = (model_pars, jobs), bounds = amdahl_par_bounds)
    return estd_amdahl_pars

## Execution

In [8]:
lam = 10
mu = 1
total_cores = 20
p_1 = 0.4
p_2 = 0.8
alpha = 0.4

amdahl_pars = [p_1, p_2]
model_pars = [lam, mu, total_cores, alpha]

n = 10000

jobs = generate_dataset(amdahl_pars, model_pars, n)
estd_amdahl_pars = estimation(model_pars, jobs)
print(estd_amdahl_pars.x)

[0.5 0.5]
[0.525 0.5  ]
[0.5   0.525]
[0.475 0.525]
[0.45   0.5375]
[0.45   0.5125]
[0.425   0.50625]
[0.375   0.54375]
[0.3125   0.565625]
[0.2875   0.534375]
[0.20625   0.5328125]
[0.09375   0.5921875]
[0.001      0.63515625]
[0.001      0.60234375]
[0.001      0.62070312]
[0.001     0.7046875]
[0.001      0.66171875]
[0.001      0.57578125]
[0.001     0.5328125]
[0.001 0.5  ]
[0.001      0.43242187]
[0.001      0.36289063]
[0.001      0.24316406]
[0.001      0.14277344]
[0.001 0.001]
[0.001 0.001]
[0.001 0.001]
[0.001 0.001]
[0.001 0.001]


In [10]:
# Likelihood of true values
LL_true = - Log_Likelihood([p_1, p_2], model_pars, jobs)
print(LL_true)

# Likelihood of estimates
LL_estd = - Log_Likelihood(estd_amdahl_pars.x, model_pars, jobs)
print(LL_estd)

LL_random = - Log_Likelihood([0.999, 0.999], model_pars, jobs)
print(LL_random)

[0.4, 0.8]
-9888.832927644082
[0.001 0.001]
-6364.2113441897345
[0.999, 0.999]
-19983.53284907046


## Sanity Check

In [11]:
# Checking whether inherent size as calculated from job history equates to true size.

for job in jobs:
    calculated_job_size = 0
    for state in job.history:
        calculated_job_size += speed_up(amdahl_pars[int(job.category)-1], state[0]) * state[1]
    print("Calculated job size = ", calculated_job_size, " , True job size = ", job.service_time,)
    
# NOTE: For some of the jobs at the end, calculated and true job sizes will not match.
# This is because they have not yet departed from system

Calculated job size =  0.11034288949096134  , True job size =  0.11034288949096134
Calculated job size =  0.4521426373451892  , True job size =  0.4521426373451892
Calculated job size =  4.534378681845895  , True job size =  4.5343786818458955
Calculated job size =  1.1485625033089573  , True job size =  1.1485625033089573
Calculated job size =  1.7260755616230647  , True job size =  1.7260755616230645
Calculated job size =  0.3918082685601685  , True job size =  0.3918082685601685
Calculated job size =  0.4655211077681732  , True job size =  0.4655211077681732
Calculated job size =  1.388194088697566  , True job size =  1.388194088697566
Calculated job size =  0.5416304897326873  , True job size =  0.5416304897326873
Calculated job size =  0.019574155138777836  , True job size =  0.019574155138777836
Calculated job size =  1.6809718965666622  , True job size =  1.6809718965666625
Calculated job size =  1.2844075573119444  , True job size =  1.2844075573119442
Calculated job size =  0.

In [ ]:
print(jobs[0].history == jobs[1].history)